In [12]:
using CategoricalArrays, Clp, CSV, DataFrames, GLPK, HiGHS, JuMP

## Challenge description

Challenge information & credit:
- The challenge was created by _Maurice Garfinkel (At AllState)_, as motivation & practice for learning Optimization techniques particular Linear Optimization (due on 2023-04-18).
- Credit goes to _Maurice Garfinkel (At AllState)_, for creating such a thoughtfully made challenge in order to start learning "Optimization Techniques".

Challenge:
- From the list of `./data/McDonalds.csv` menu items choose items for 30 days that minimize the total calories while meeting the limits in `./data/nutrient_limits.csv`.
    - Furthermore:
        - A menu item can be chosen only once in a day (i.e. In one day cannot choose the same menu item twice). 
        - The meal for each day needs to have at least 3 mains from either {"Entrees", "Burgers", "Sandwiches"}.
        - Fractional meals are okay.
- Show that the solution has a value of `54983.2`.

## Intuition/motivation for the solution

1. First note that this is an optimization problem, since there are multiple combinations McDonalds meals for 30 days, and we need to pick an optimal combination subject to the calory & nutrient constraints.
2. This is a "Linear Programming" problem, since "Linear Programming" problems are special class of Optimization problems where the "requirements" are represented by linear relationships.

Therefore, since "Optimization" & "Linear Programming" are well established problem solving techiniques for exactly these kinds of problems, we should at least first start to solve with these.

Reference:
- [Wikipedia: Linear Programming](https://en.wikipedia.org/wiki/Linear_programming)

## Get data

In [2]:
df_mcdonalds_menu = CSV.read("./data/McDonalds.csv", DataFrame)
df_nutrient_limits = CSV.read("./data/nutrient_limits.csv", DataFrame)

Row,Nutrient,Lower_Limit,Upper_Limit
,String15,Int64?,Float64
1,Total_Fat,70,90.0
2,Saturated_Fat,20,30.0
3,Trans_Fat,missing,2.75
4,Cholesterol,missing,300.0
5,Sodium,missing,2300.0
6,Carbohydrates,225,400.0
7,Protein,50,220.0
8,Sugar,missing,36.0
9,Fiber,25,38.0


## Briefly look at the data

### McDonalds menu

In [3]:
first(df_mcdonalds_menu, 2)

Row,Restaurant_Item_Name,Food_Category,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Cholesterol,Sodium,Carbohydrates,Protein,Sugar,Dietary_Fiber
,String,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,McDonald's Dr Pepper Large,Beverages,290.0,0.0,0.0,0.0,0.0,95.0,79.0,0.0,78.0,0.0
2,McDonald's Dr Pepper Medium,Beverages,210.0,0.0,0.0,0.0,0.0,70.0,58.0,0.0,57.0,0.0


In [4]:
# Check if Calories are unique

print(nrow(df_mcdonalds_menu))
length(unique(df_mcdonalds_menu.Calories))

443

96

### nutrient_limits

In [5]:
df_nutrient_limits

Row,Nutrient,Lower_Limit,Upper_Limit
,String15,Int64?,Float64
1,Total_Fat,70,90.0
2,Saturated_Fat,20,30.0
3,Trans_Fat,missing,2.75
4,Cholesterol,missing,300.0
5,Sodium,missing,2300.0
6,Carbohydrates,225,400.0
7,Protein,50,220.0
8,Sugar,missing,36.0
9,Fiber,25,38.0


## Feature engineering

### Rename columns

In [6]:
# Rename Dietary_Fiber column to match name in nutrient_limits

rename!(df_mcdonalds_menu, :Dietary_Fiber => :Fiber)

Row,Restaurant_Item_Name,Food_Category,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Cholesterol,Sodium,Carbohydrates,Protein,Sugar,Fiber
,String,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,McDonald's Dr Pepper Large,Beverages,290.0,0.0,0.0,0.0,0.0,95.0,79.0,0.0,78.0,0.0
2,McDonald's Dr Pepper Medium,Beverages,210.0,0.0,0.0,0.0,0.0,70.0,58.0,0.0,57.0,0.0
3,McDonald's Tangy Barbeque Sauce,Toppings & Ingredients,45.0,0.0,0.0,0.0,0.0,260.0,11.0,0.0,9.0,0.0
4,McDonald's Sweet N Sour Sauce,Toppings & Ingredients,50.0,0.0,0.0,0.0,0.0,160.0,11.0,1.0,9.0,0.0
5,McDonald's Hamburger,Burgers,250.0,8.0,3.0,0.0,30.0,480.0,31.0,13.0,6.0,2.0
6,McDonald's Cheeseburger,Burgers,300.0,12.0,5.0,0.5,40.0,680.0,33.0,15.0,7.0,2.0
7,McDonald's Double Cheeseburger,Burgers,430.0,21.0,11.0,1.0,85.0,1040.0,35.0,25.0,7.0,2.0
8,McDonald's Quarter Pounder w/ Cheese,Burgers,530.0,27.0,13.0,1.5,100.0,1090.0,41.0,31.0,10.0,3.0
9,McDonald's Big Mac,Burgers,540.0,28.0,10.0,1.0,80.0,950.0,46.0,25.0,9.0,3.0


### Convert categorical data to the CategoricalArray data type

In [7]:
df_mcdonalds_menu.Restaurant_Item_Name = categorical(df_mcdonalds_menu.Restaurant_Item_Name)
df_mcdonalds_menu.Food_Category = categorical(df_mcdonalds_menu.Food_Category)

df_nutrient_limits.Nutrient = categorical(df_nutrient_limits.Nutrient)

# Add numeric versions
df_mcdonalds_menu.Restaurant_Item_Name_Num = levelcode.(df_mcdonalds_menu.Restaurant_Item_Name)
df_mcdonalds_menu.Food_Category_Num = levelcode.(df_mcdonalds_menu.Food_Category)

df_nutrient_limits.Nutrient_Num = levelcode.(df_nutrient_limits.Nutrient)


9-element Vector{Int64}:
 8
 5
 9
 2
 6
 1
 4
 7
 3

### Fill in missing data

In [8]:
df_nutrient_limits.Lower_Limit = coalesce.(df_nutrient_limits.Lower_Limit, 0)

9-element Vector{Int64}:
  70
  20
   0
   0
   0
 225
  50
   0
  25

## Applying Linear Optimization 

In [9]:
df_mcdonalds_menu__mains = filter((x) -> x.Food_Category in ["Entrees", "Burgers", "Sandwiches"], df_mcdonalds_menu)
df_mcdonalds_menu__non_main = filter((x) -> !(x.Food_Category in ["Entrees", "Burgers", "Sandwiches"]), df_mcdonalds_menu)

Row,Restaurant_Item_Name,Food_Category,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Cholesterol,Sodium,Carbohydrates,Protein,Sugar,Fiber,Restaurant_Item_Name_Num,Food_Category_Num
,Cat…,Cat…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,McDonald's Dr Pepper Large,Beverages,290.0,0.0,0.0,0.0,0.0,95.0,79.0,0.0,78.0,0.0,45,2
2,McDonald's Dr Pepper Medium,Beverages,210.0,0.0,0.0,0.0,0.0,70.0,58.0,0.0,57.0,0.0,46,2
3,McDonald's Tangy Barbeque Sauce,Toppings & Ingredients,45.0,0.0,0.0,0.0,0.0,260.0,11.0,0.0,9.0,0.0,391,9
4,McDonald's Sweet N Sour Sauce,Toppings & Ingredients,50.0,0.0,0.0,0.0,0.0,160.0,11.0,1.0,9.0,0.0,386,9
5,McDonald's French Fries Small,Fried Potatoes,230.0,11.0,1.5,0.0,0.0,160.0,29.0,3.0,0.0,3.0,97,6
6,McDonald's French Fries Medium,Fried Potatoes,340.0,16.0,2.0,0.0,0.0,230.0,44.0,4.0,0.0,4.0,96,6
7,McDonald's French Fries Large,Fried Potatoes,510.0,24.0,3.5,0.0,0.0,350.0,66.0,7.0,0.0,6.0,95,6
8,McDonald's Bacon Ranch Salad w/ Grilled Chicken,Salads,320.0,14.0,6.0,0.0,135.0,1090.0,9.0,42.0,4.0,3.0,23,7
9,McDonald's Bacon Ranch Salad w/ Buttermilk Crispy Chicken,Salads,490.0,28.0,8.0,0.0,95.0,1120.0,28.0,33.0,4.0,4.0,22,7


In [19]:
model = Model(HiGHS.Optimizer)
set_silent(model)

In [35]:
# debug

nonunique(df_mcdonalds_menu[!, ["Restaurant_Item_Name"]])

443-element Vector{Bool}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 1
 1
 0
 1
 0
 0
 0
 0
 1
 0
 0
 0

In [34]:
# debug

sort(df_mcdonalds_menu[nonunique(df_mcdonalds_menu[!, ["Restaurant_Item_Name"]]), "Restaurant_Item_Name"])

43-element CategoricalArray{String,1,UInt32}:
 "McDonald's 20 Chicken McNuggets"
 "McDonald's Artisan Grilled Chicken Sandwich"
 "McDonald's Baked Hot Apple Pie"
 "McDonald's Big Breakfast w/ Hotcakes"
 "McDonald's Coca Cola Extra Small"
 "McDonald's Coca Cola Large"
 "McDonald's Coca Cola Medium"
 "McDonald's Coca Cola Small"
 "McDonald's Dr Pepper Small"
 "McDonald's Egg McMuffin"
 "McDonald's Egg White Delight McMuffin"
 "McDonald's Extra Value Meal w/ Bacon Egg & Cheese Biscuit"
 "McDonald's Extra Value Meal w/ Bacon Egg & Cheese McGriddles"
 ⋮
 "McDonald's Powerade Mountain Berry Blast Medium"
 "McDonald's Sausage McMuffin"
 "McDonald's Sausage McMuffin w/ Egg"
 "McDonald's Sprite Extra Small"
 "McDonald's Sprite Large"
 "McDonald's Sprite Medium"
 "McDonald's Sprite Small"
 "McDonald's Strawberry & Cream Pie"
 "McDonald's Strawberry Sundae"
 "McDonald's Sweet N Sour Sauce"
 "McDonald's Sweet Tea Extra Small"
 "McDonald's Tangy Barbeque Sauce"

In [33]:
# debug

sort(df_mcdonalds_menu[findall(nonunique(df_mcdonalds_menu[!, ["Restaurant_Item_Name"]])), "Restaurant_Item_Name"])

43-element CategoricalArray{String,1,UInt32}:
 "McDonald's 20 Chicken McNuggets"
 "McDonald's Artisan Grilled Chicken Sandwich"
 "McDonald's Baked Hot Apple Pie"
 "McDonald's Big Breakfast w/ Hotcakes"
 "McDonald's Coca Cola Extra Small"
 "McDonald's Coca Cola Large"
 "McDonald's Coca Cola Medium"
 "McDonald's Coca Cola Small"
 "McDonald's Dr Pepper Small"
 "McDonald's Egg McMuffin"
 "McDonald's Egg White Delight McMuffin"
 "McDonald's Extra Value Meal w/ Bacon Egg & Cheese Biscuit"
 "McDonald's Extra Value Meal w/ Bacon Egg & Cheese McGriddles"
 ⋮
 "McDonald's Powerade Mountain Berry Blast Medium"
 "McDonald's Sausage McMuffin"
 "McDonald's Sausage McMuffin w/ Egg"
 "McDonald's Sprite Extra Small"
 "McDonald's Sprite Large"
 "McDonald's Sprite Medium"
 "McDonald's Sprite Small"
 "McDonald's Strawberry & Cream Pie"
 "McDonald's Strawberry Sundae"
 "McDonald's Sweet N Sour Sauce"
 "McDonald's Sweet Tea Extra Small"
 "McDonald's Tangy Barbeque Sauce"

In [22]:
# debug

df_mcdonalds_menu[findall(nonunique(df_mcdonalds_menu.Restaurant_Item_Name)), :]

LoadError: MethodError: no method matching nonunique(::CategoricalVector{String, UInt32, String, CategoricalValue{String, UInt32}, Union{}})
[0mClosest candidates are:
[0m  nonunique([91m::AbstractDataFrame[39m; keep) at ~/.julia/packages/DataFrames/LteEl/src/abstractdataframe/unique.jl:86
[0m  nonunique([91m::AbstractDataFrame[39m, [91m::Any[39m; keep) at ~/.julia/packages/DataFrames/LteEl/src/abstractdataframe/unique.jl:156

In [17]:
# debug

df_mcdonalds_menu__mains[findall(nonunique(df_mcdonalds_menu__mains)), :]

Row,Restaurant_Item_Name,Food_Category,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Cholesterol,Sodium,Carbohydrates,Protein,Sugar,Fiber,Restaurant_Item_Name_Num,Food_Category_Num
,Cat…,Cat…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64


In [18]:
# debug

df_mcdonalds_menu__non_main[findall(nonunique(df_mcdonalds_menu__non_main)), :]

Row,Restaurant_Item_Name,Food_Category,Calories,Total_Fat,Saturated_Fat,Trans_Fat,Cholesterol,Sodium,Carbohydrates,Protein,Sugar,Fiber,Restaurant_Item_Name_Num,Food_Category_Num
,Cat…,Cat…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64


In [20]:
@variable(model, xm[df_mcdonalds_menu__mains.Restaurant_Item_Name] >= 0)
@variable(model, xnm[df_mcdonalds_menu__non_mains.Restaurant_Item_Name] >= 0)

LoadError: Repeated index McDonald's Artisan Grilled Chicken Sandwich. Index sets must have unique elements.

----

In [ ]:
n_mains = nrow(df_mcdonalds_menu__mains)
n_non_main = nrow(df_mcdonalds_menu__non_main)

@variable(model, xm[1:n_mains] >= 0)
@variable(model, xnm[1:n_non_main] >= 0)
@variable(model, constraint_mains[1:n_mains], Bin)

In [ ]:
@objective(model, Min, sum(df_mcdonalds_menu_.Calories[i]*x[i] for i in 1:n))

----

Questions for Maurice:
- Is creating variables in a for loop for the 30 days the best way to the optimization equation? This way you end up with at least `30*n_menu_items` number of variables 

----

In [ ]:
domain_menu_variables = unique(
    select(
        df_mcdonalds_menu, 
        :Restaurant_Item_Name_Num => minimum => :item_min,
        :Restaurant_Item_Name_Num => maximum => :item_max,
        :Food_Category_Num => minimum => :food_category_min,
        :Food_Category_Num => maximum => :food_category_max,
    )
)

In [ ]:
domain_menu_variables[!, r"min"]

In [ ]:
model = Model()
S = [2, 5, 6, 8, 10]
@variable(model, x, Int)
@variable(model, z[1:length(S)], Bin)
@constraint(model, sum(z) == 1)
@constraint(model, x == sum(S[i] * z[i] for i = 1:length(S))